In [ ]:
!pip install lightning 
!pip install comet_ml

In [17]:
import os

import lightning as L
import lightning.pytorch as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning.pytorch.callbacks import StochasticWeightAveraging, ModelCheckpoint
from pytorch_lightning.loggers import CometLogger
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset

# Setting the seed
L.seed_everything(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Global seed set to 42


Device: cpu


In [18]:
LARGE_CONSTANT=2**8

In [19]:
class DivisionDataset(Dataset):
    def __init__(self, upper_limit, dataset_size, seed=None):
        self.upper_limit = upper_limit
        self.dataset_size = dataset_size
        if seed!=None: torch.manual_seed(42)
        self.data = (torch.rand(self.dataset_size))*self.upper_limit+1
        self.data = torch.cat((torch.rand(dataset_size)+0.01,self.data))
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        rnd_num = self.data[idx]
        return (rnd_num), (1/rnd_num)

In [20]:
class DivisionDatasetZeroOne(Dataset):
    def __init__(self, upper_limit, dataset_size, seed=None):
        self.upper_limit = upper_limit
        self.dataset_size = dataset_size
        if seed!=None: torch.manual_seed(42)
        self.data = (torch.rand(dataset_size)+0.1)
    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        rnd_num = self.data[idx]
        return (rnd_num), (1/rnd_num)

In [21]:
DATASET_SIZE = 2**14
UPPER_LIMIT = 256
BATCH_SIZE = 32
train_dataloader = DataLoader(DivisionDataset(UPPER_LIMIT,DATASET_SIZE),batch_size=BATCH_SIZE,shuffle=True)
val_dataloader = DataLoader(DivisionDataset(UPPER_LIMIT,DATASET_SIZE),batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = DataLoader(DivisionDataset(UPPER_LIMIT,DATASET_SIZE),batch_size=BATCH_SIZE,shuffle=True)

In [22]:
# next(iter(train_dataloader))[0]+1

In [23]:
# train_dataloader_zeroone = DataLoader(DivisionDatasetZeroOne(16,DATASET_SIZE),batch_size=BATCH_SIZE)
# val_dataloader_zeroone = DataLoader(DivisionDatasetZeroOne(16,DATASET_SIZE),batch_size=BATCH_SIZE)

In [24]:
# next(iter(train_dataloader_zeroone))[1]*LARGE_CONSTANT

In [25]:
class SqrtModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(1, 128), 
                                nn.ReLU(),
                                nn.Linear(128, 256),
                                nn.ReLU(),
                                nn.Linear(256, 1),
                                )

    def forward(self, x):
        return self.l1(x)

class SqrtLightning(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = SqrtModel()
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.tcounter = 0
        self.vcounter = 0
        
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        y = y.view(y.size(0), -1)
        x_hat = self.model(x)
        loss = F.mse_loss(x_hat, y)
        self.training_step_outputs.append(torch.mean(torch.abs(x_hat-y)))
        return loss

    def on_train_epoch_end(self):
        loss = torch.mean(torch.Tensor(self.training_step_outputs))
        self.logger.log_metrics({"train_epoch_loss": loss},step=self.tcounter)
        self.tcounter+=1
        self.training_step_outputs.clear()

    def validation_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        y = y.view(y.size(0), -1)
        x_hat = self.model(x)
        loss = F.mse_loss(x_hat, y)
        self.validation_step_outputs.append(torch.mean(torch.abs(x_hat-y)))

    def on_validation_epoch_end(self):
        loss = torch.mean(torch.Tensor(self.validation_step_outputs))
        self.logger.log_metrics({"val_epoch_loss": loss},step=self.vcounter)
        self.log("val_epoch_loss_metric", loss)
        self.vcounter+=1
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=4)
        # scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-2,cycle_momentum=False)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_epoch_loss_metric",
            },
        }

In [26]:
comet_logger = CometLogger(api_key = "nFCjgGBWWw18R4icR55qjnYfO",project_name="sqrtexperiments",workspace="hovhannesmanushyan")


CometLogger will be initialized in online mode


In [27]:
checkpoint_callback = ModelCheckpoint(
    dirpath=os.getcwd(),
    save_top_k=1,
    verbose=True,
    monitor='val_epoch_loss_metric',
    mode='min',
)

In [28]:
sqrt_model = SqrtLightning()

# train model
# trainer = pl.Trainer(accelerator="gpu",max_epochs=100,logger=comet_logger,callbacks = [StochasticWeightAveraging(swa_lrs=1e-5)], gradient_clip_val=0.5)
trainer = pl.Trainer(reload_dataloaders_every_n_epochs=1, accelerator="gpu",max_epochs=100,logger=comet_logger,callbacks = [checkpoint_callback,StochasticWeightAveraging(swa_lrs=1e-5)], gradient_clip_val=0.5)
trainer.fit(model=sqrt_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
comet_logger.experiment.log_model("division_model", checkpoint_callback.best_model_path)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/elinaisrayelyan/PycharmProjects/capstone_project/venv/lib/python3.9/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/elinaisrayelyan/PycharmProjects/capstone_project/notebooks exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name  | Type      | Params
------------------------------------
0 | model | SqrtModel | 33.5 K
------------------------------------
33.5 K    Trainable params
0         Non-trainable params
33.5 K    Total params
0.134     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/elinaisrayelyan/PycharmProjects/capstone_project/venv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/Users/elinaisrayelyan/PycharmProjects/capstone_project/venv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metr

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 1024: 'val_epoch_loss_metric' reached 2.35002 (best 2.35002), saving model to '/Users/elinaisrayelyan/PycharmProjects/capstone_project/notebooks/epoch=0-step=1024.ckpt' as top 1
/Users/elinaisrayelyan/PycharmProjects/capstone_project/venv/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/hovhannesmanushyan/sqrtexperiments/358e48e40fe8430a8646092ec7fd68b8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_epoch_loss      : 2.43111

{'web': 'https://www.comet.com/api/asset/download?assetId=0b85137de2c44e88b80b9ec5b133969b&experimentKey=358e48e40fe8430a8646092ec7fd68b8',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=0b85137de2c44e88b80b9ec5b133969b&experimentKey=358e48e40fe8430a8646092ec7fd68b8',
 'assetId': '0b85137de2c44e88b80b9ec5b133969b'}

In [182]:
model = SqrtLightning().load_from_checkpoint("../epoch=94-step=97280.ckpt")
# model = SqrtLightning().load_from_checkpoint("/content/epoch=43-step=45056.ckpt")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [15]:
model.eval()

NameError: name 'model' is not defined

In [ ]:
model.model(torch.Tensor([0.04]).to("cuda:0"))

In [ ]:
1/64

In [16]:
!rm -rf *.ckpt

zsh:1: no matches found: *.ckpt


In [ ]:
1/5

# Encryption


In [116]:
import sys

sys.path.insert(0, '/Users/elinaisrayelyan/PycharmProjects/capstone_project')
import torch.nn as nn

import torch
import tenseal as ts


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
image_size = 20  # poly degree /2 should be bigger than image to columns for conv size
# context, server_context = create_ctx(bits_scale=33,poly_mod_degree=32768, num_mul=9)  # scale up as working with floats,
import tenseal as ts


def create_ctx(bits_scale=40, poly_mod_degree=16384, num_mul=5):
    """Helper for creating the CKKS context.
    CKKS params:
        - Polynomial degree: 8192.
        - Coefficient modulus size: [40, 21, 21, 21, 21, 21, 21, 40]. # 24 binary digit
        - Scale: 2 ** 21. # 24 ov kara
        - The setup requires the Galois keys for evaluating the convolutions.
    """

    coeff_mod_bit_sizes = [40]
    for i in range(num_mul):
        coeff_mod_bit_sizes.append(bits_scale)
    coeff_mod_bit_sizes.append(40)

    ctx = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    ctx.global_scale = pow(2, bits_scale)
    ctx.generate_galois_keys()

    # We prepare the context for the server, by making it public(we drop the secret key)
    server_context = ctx.copy()
    server_context.make_context_public()

    return ctx, server_context

context, server_context = create_ctx(bits_scale=31,poly_mod_degree=32768//2,num_mul=9)

In [176]:

class EncDivNet:

    def __init__(self, sqrt_model):

        self.fc1_weight = sqrt_model.model.l1[0].weight.T.data
        self.fc1_bias = sqrt_model.model.l1[0].bias.data.T

        self.fc2_weight = sqrt_model.model.l1[2].weight.T.data
        self.fc2_bias = sqrt_model.model.l1[2].bias.data.T

        self.fc3_weight = sqrt_model.model.l1[4].weight.T.data
        self.fc3_bias = sqrt_model.model.l1[4].bias.data.T
    def forward(self, enc_x):

        # pack all channels into a single flattened vector
        # enc_x = ts.CKKSVector.pack_vectors(enc_channels)
        # fc1 layer
        enc_x = enc_x.mm(self.fc1_weight) + self.fc1_bias

        enc_x = self.relu_n(enc_x)
        # fc2 layer
        enc_x = enc_x.mm(self.fc2_weight) + self.fc2_bias
        enc_x = self.relu_n(enc_x)

        enc_x = enc_x.mm(self.fc3_weight) + self.fc3_bias

        return enc_x

    @staticmethod
    def relu(enc_x):
        # We use the polynomial approximation of degree 3
        # relu(x) = 0.47+0.5*x+0.09*(x**2)-1.7*e-10*(x**3)
        # from https://openreview.net/attachment?id=rkxsgkHKvH&name=original_pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.47, 0.5, 0.09, -0.0000000017])

    @staticmethod
    def relu_n(enc_x):
        return enc_x.polyval([1, 0.5, 5e-14])
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [177]:
enc_div  = EncDivNet(sqrt_model)

In [178]:
enc_x = ts.ckks_vector(context,[2])

In [179]:
enc_x_div = enc_div(enc_x)

In [180]:
enc_x_div.decrypt()

[-16.5052220729374]

In [174]:
sqrt_model.model.l1(torch.tensor([2.0]))

tensor([0.1462], grad_fn=<AddBackward0>)

In [175]:
sqrt_model.model.l1[1](sqrt_model.model.l1[0](torch.tensor([2.0])))

tensor([1.8557, 0.0000, 0.0000, 0.0000, 0.5405, 2.3742, 2.1246, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 2.0941, 0.0000, 2.7686, 0.0000, 2.7671, 1.9660,
        0.0000, 1.4941, 0.9698, 1.0321, 1.3382, 0.0000, 2.5431, 0.0000, 1.9721,
        0.2645, 2.6354, 0.0000, 0.0000, 0.0000, 0.0000, 0.6435, 0.0000, 1.8542,
        0.0000, 1.1287, 0.0000, 0.0000, 1.5587, 0.0000, 0.1321, 2.4943, 0.0000,
        0.0000, 2.0257, 0.4901, 0.0000, 2.3163, 1.9737, 0.0000, 0.9601, 1.2625,
        0.0000, 0.0000, 1.3527, 0.0000, 0.0000, 0.4548, 0.0000, 0.0000, 1.2367,
        0.0000, 1.4659, 0.0000, 0.0000, 0.0000, 0.0000, 1.9974, 0.0000, 1.5847,
        0.0000, 1.5270, 0.7764, 0.0000, 2.3249, 0.0000, 0.0000, 0.3057, 2.6565,
        0.0000, 0.0000, 1.1150, 1.0345, 1.2710, 0.0000, 0.0000, 0.0000, 0.0000,
        0.6961, 0.0000, 0.0000, 0.0000, 1.0342, 0.7110, 1.1716, 0.0000, 0.0000,
        0.0000, 2.0082, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.2699, 0.0000,
        0.0000, 0.0000, 2.5134, 0.8540, 